In [1]:
!pip install -q findspark

In [7]:
%load_ext jupyter_black

In [8]:
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk"
os.environ["SPARK_HOME"] = f'{os.environ["HOME"]}/content/spark-3.5.1-bin-hadoop3'
import findspark

findspark.init()

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()
spark.conf.set(
    "spark.sql.repl.eagerEval.enabled", True
)  # Property used to format output tables better
sc = spark.sparkContext

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/05/25 15:19:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [11]:
# Exercise 1. Text file manipulations
content_dir = f'{os.environ["HOME"]}/content'

addr = f"{content_dir}/alice_in_wonderland.txt"
addrTarget = f"{content_dir}/rabbit"

aliceRDD = sc.textFile(addr)

# aliceRDD.take(5)

rdd10 = aliceRDD.take(10)

for line in rdd10:
    print(line)

rabbitRDD = aliceRDD.map(lambda x: x.lower()).filter(lambda x: "rabbit" in x)

# rabbitRDD.collect()
rabbitRDD.count()
rabbitRDD.saveAsTextFile(addrTarget)

Alice's Adventures in Wonderland

                ALICE'S ADVENTURES IN WONDERLAND

                          Lewis Carroll

               THE MILLENNIUM FULCRUM EDITION 3.0





In [12]:
# Exercise 2. User-defined functions in transformations
addr = f'{content_dir}/kv.txt'

kv_RDD = sc.textFile(addr)

#kv_RDD.take(10)

kv_separated_RDD = kv_RDD \
              .map(lambda line: line.split('\x01'))

kv_separated_RDD.take(10)

def replacement(l):
    return [l[0], l[1].replace('val_', 'value is ')]

def even_check(l):
  return l[0]%2==0

def transformColl(coll):
    for item in coll:
        for el in item:
            if type(el) is int:
                print(el + 1000)
            elif type(el) is str:
                print(el * 2)
            else:
                print("ERROR")

even_RDD = kv_separated_RDD \
              .map(replacement) \
              .map(lambda line: [line[0], line[1][9:]]) \
              .map(lambda line: (int(line[0]), line[1])) \
              .filter(even_check) \
              .filter(lambda line: line[0] != 0)

transformColl(even_RDD.take(10))

Py4JJavaError: An error occurred while calling o63.partitions.
: org.apache.hadoop.mapred.InvalidInputException: Input path does not exist: file:/home/succ_seed/content/kv.txt
	at org.apache.hadoop.mapred.FileInputFormat.singleThreadedListStatus(FileInputFormat.java:304)
	at org.apache.hadoop.mapred.FileInputFormat.listStatus(FileInputFormat.java:244)
	at org.apache.hadoop.mapred.FileInputFormat.getSplits(FileInputFormat.java:332)
	at org.apache.spark.rdd.HadoopRDD.getPartitions(HadoopRDD.scala:208)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.rdd.MapPartitionsRDD.getPartitions(MapPartitionsRDD.scala:49)
	at org.apache.spark.rdd.RDD.$anonfun$partitions$2(RDD.scala:294)
	at scala.Option.getOrElse(Option.scala:189)
	at org.apache.spark.rdd.RDD.partitions(RDD.scala:290)
	at org.apache.spark.api.java.JavaRDDLike.partitions(JavaRDDLike.scala:61)
	at o

In [ ]:
# Exercise 2. User-defined functions in transformations
addr = '/content/kv.txt'

kvRDD = sc.textFile(addr)
#kvRDD.take(10)

#kvRDD.map(lambda x: x.split("\x01")).take(10)

def replaceValue(l):
    return [l[0], l[1].replace( "val_", "value is ")]

#kvRDD.map(lambda x: x.split("\x01")).map(replaceValue).take(10)

def cut08(s):
    return s[9:]

def checkEven(x):
    return x%2==0

def transformColl(coll):
    for item in coll:
        for el in item:
            if type(el) is int:
                print(el + 1000)
            elif type(el) is str:
                print(el * 2)
            else:
                print("ERROR")


#kvRDD.map(lambda x: x.split("\x01")).map(replaceValue).map(lambda x:[x[0],cut08(x[1])]).take(5)

"""
kvRDD.map(lambda x: x.split("\x01")).map(replaceValue)\
.map(lambda x:[x[0],cut08(x[1])])\
.map(lambda x: (int(x[0]), x[1])).take(5)
"""

tRDD = kvRDD.map(lambda x: x.split("\x01")).map(replaceValue)\
.map(lambda x:[x[0],cut08(x[1])])\
.map(lambda x: (int(x[0]), x[1]))

eRDD = tRDD.filter(lambda x: checkEven(x[0]))

#eRDD.take(10)

c = eRDD.take(20)
transformColl(c)

1238
238238
1086
8686
1278
278278
1098
9898
1484
484484
1150
150150
1224
224224
1066
6666
1128
128128
1146
146146
1406
406406
1374
374374
1152
152152
1082
8282
1166
166166
1430
430430
1252
252252
1292
292292
1338
338338
1446
446446


In [ ]:
# Exercise 3. Text processing

addr = '/content/alice_in_wonderland.txt'

aliceRDD = sc.textFile(addr)

def capitalise_word(line):
    new_string = ''
    for word in line.split(' '):
      if len(word) > 1:
        #new_string += word[0].upper()+word[1:].lower()+' '
        new_string += word.capitalize()+' '
    return new_string

def twist_word(line):
    new_string = ''
    for word in line.split(' '):
      if len(word) > 1:
        new_string += word[0].lower()+word[1:].upper()+' '
    return new_string

changed_alice_RDD = aliceRDD.map(twist_word)

changed_alice_RDD.take(5)


["aLICE'S aDVENTURES iN wONDERLAND ",
 '',
 "aLICE'S aDVENTURES iN wONDERLAND ",
 '',
 'lEWIS cARROLL ']

In [ ]:
# Exercise 3. Text processing

def capWordsInLine(line):
    res = ""
    w = ""
    for word in line.split(" "):
        w = word.upper()
        if len(w) > 1:
            res += w[0].upper() + w[1:].lower() + " "
    return res

def capWordsInLine2(line):
    res = ""
    w = ""
    for word in line.split(" "):
        if len(word) > 1:
            res += word.capitalize() + " "
    return res

def capWordsInLine3(line):
    res = ""
    w = ""
    for word in line.split(" "):
        w = word.lower()
        if len(w) > 1:
            res += w[0] + w[1:].upper() + " "
    return res

addr = '/content/alice_in_wonderland.txt'

aliceRDD = sc.textFile(addr)

#aliceRDD.map(capWordsInLine).take(10)

#aliceRDD.map(capWordsInLine2).take(10)

aliceRDD.map(capWordsInLine3).take(10)



["Alice's Adventures In Wonderland ",
 '',
 "Alice's Adventures In Wonderland ",
 '',
 'Lewis Carroll ',
 '',
 'The Millennium Fulcrum Edition 3.0 ',
 '',
 '',
 '']